# Plot convergence curves

In [60]:
import matplotlib as mpb

# show all the matplotlib backends 
mpb.rcsetup.all_backends 

# force matplotlib to use the 'Qt5Agg' backend 
mpb.use(arg='Qt5Agg', force=True)

In [61]:
import numpy as np
import matplotlib.pyplot as plt
import cPickle as pickle
import scipy.stats as sp
import os

from fatiando.gravmag import polyprism
from mpl_toolkits.mplot3d import Axes3D
from mpl_toolkits.mplot3d.art3d import Poly3DCollection
from fatiando.vis import mpl
from mpl_toolkits.axes_grid1.inset_locator import inset_axes

In [62]:
import sys
sys.path.insert(0, '../../code')

import mag_polyprism_functions as mfun
import mag_sphere_functions as msf

In [63]:
mypath = 'tfa_inversion/upward_multiple_int_min3_int_max6_z0_min-900_z0_max-750_r1500_dz600_V30_L10Feb-06-2020/'

In [64]:
with open(mypath+'inversion.pickle') as w:
        di = pickle.load(w)

In [65]:
plt.figure(figsize=(10,10))
plt.title('Convergence', fontsize = 18)
plt.xlabel('Iteration number')
for i, phi in enumerate(di['results']):
    plt.plot(phi[3], label='inversion%d' % (i+1))
plt.legend(loc=0, fontsize=2.5, shadow=bool, framealpha=1)

plt.savefig(mypath+'_tfa_convergence.png', dpi=300)
plt.show()

In [66]:
z0 = di['z0']
intensity = di['intensity']
z0_plot, int_plot = np.meshgrid(z0, intensity)
z0_plot = z0_plot.ravel()
int_plot = int_plot.ravel()
n = z0.size
m = intensity.size
N = di['x'].size

In [67]:
matrix_gamma = np.zeros((n,m))
for i in range(n):
    for j in range(m):
        matrix_gamma[i, j] = di['results'][i*n+j][3][-1]

In [68]:
matrix_phi = np.zeros((n,m))
for i in range(n):
    for j in range(m):
        matrix_phi[i, j] = np.linalg.norm(di['results'][i*n+j][6][-1], 2)**2./N

In [69]:
mindex = np.argwhere(matrix_gamma<60.)
mindex

array([[0, 2],
       [1, 3],
       [2, 3],
       [2, 4],
       [3, 4],
       [4, 5]])

In [70]:
dots_z0 = []
dots_int = []
for re in mindex:
    print re[0]*10 + re[1]
    print matrix_gamma[re[0], re[1]]
    dots_int.append(np.linalg.norm(di['results'][re[0]*n + re[1]][2][0].props['magnetization']))
    dots_z0.append(di['results'][re[0]*n + re[1]][2][0].z1)

2
52.09623318713226
13
51.0955405846368
23
57.93191893209328
24
57.79674708043074
34
56.41521633763493
45
57.60944370440586


In [71]:
misdex = np.argwhere(matrix_phi<60.)
misdex

array([[0, 2],
       [2, 4],
       [4, 5]])

In [72]:
for re in misdex:
    print re[0]*10 + re[1]
    print matrix_phi[re[0], re[1]]

2
48.08842032580277
24
55.471704704031666
45
54.255309415487645


In [73]:
fig, ax = fig, ax = plt.subplots(1,1)
img = ax.imshow(matrix_gamma, vmin=40, vmax=100, origin='lower',extent=[0,1,0,1])
plt.ylabel('$z_0$ (m)')
plt.xlabel('$m_0$ (A/m)')
x_label_list = []
y_label_list = []
for xl, yl in zip(intensity,z0):
    x_label_list.append(str(xl))
    y_label_list.append(str(yl)[:-2])
ax.set_xticks(np.linspace(1./(2.*n), 1 - 1./(2.*n), n))
ax.set_yticks(np.linspace(1./(2.*m), 1 - 1./(2.*m), m))
ax.set_xticklabels(x_label_list)
ax.set_yticklabels(y_label_list)
# Minor ticks
ax.set_xticks(np.linspace(0, 1, n+1), minor=True);
ax.set_yticks(np.linspace(0, 1, m+1), minor=True);
ax.grid(which='minor', color='k', linewidth=1)
plt.colorbar(img)
plt.show()

In [74]:
fig, ax = fig, ax = plt.subplots(1,1)
img = ax.imshow(matrix_phi, vmin=40, vmax=100, origin='lower',extent=[0,1,0,1], aspect='equal')
plt.ylabel('$z_0$ (m)')
plt.xlabel('$m_0$ (A/m)')
x_label_list = []
y_label_list = []
for xl, yl in zip(intensity,z0):
    x_label_list.append(str(xl))
    y_label_list.append(str(yl)[:-2])
ax.set_xticks(np.linspace(1./(2.*n), 1 - 1./(2.*n), n))
ax.set_yticks(np.linspace(1./(2.*m), 1 - 1./(2.*m), m))
ax.set_xticklabels(x_label_list)
ax.set_yticklabels(y_label_list)
# Minor ticks
ax.set_xticks(np.linspace(0, 1, n+1), minor=True);
ax.set_yticks(np.linspace(0, 1, m+1), minor=True);
ax.grid(which='minor', color='k', linewidth=1)
plt.colorbar(img)
plt.show()

# Inversion 13

In [58]:
for re in mindex:

    plt.figure(figsize=(6,5))
    ax=mpl.subplot(1,1,1)
    #plt.title('Residual', fontsize=20)
    norm=mpb.colors.Normalize(vmin=np.min(di['results'][re[0]*10 + re[1]][-2]), vmax=np.min(di['results'][re[0]*10 + re[1]][-2])*(-1))
    plt.tricontourf(di['y'], di['x'], di['results'][re[0]*10 + re[1]][-2], 20, cmap='RdBu_r', norm=norm).ax.tick_params(labelsize=12)
    plt.xlabel('$y$(m)', fontsize=18)
    plt.ylabel('$x$(m)', fontsize=18)
    clb = plt.colorbar(pad=0.01, aspect=20, shrink=1)
    clb.ax.set_title('nT')
    mpl.m2km()
    clb.ax.tick_params(labelsize=13)
    inset = inset_axes(ax, width="40%", height="30%", loc=1, borderpad=0.7)
    mean = np.mean(di['results'][re[0]*10 + re[1]][-2])
    std = np.std(di['results'][re[0]*10 + re[1]][-2])
    nbins=30
    n, bins, patches = plt.hist(di['results'][re[0]*10 + re[1]][-2],bins=nbins, normed=True, facecolor='blue')
    plt.text(0.4, 0.96, "$\mu$ = {:.2f}\n$\sigma$ = {:.2f}".format(mean, std), 
             horizontalalignment='right',
             verticalalignment='top',
             transform = ax.transAxes, fontsize=10)
    gauss = sp.norm.pdf(bins, mean, std)
    plt.plot(bins, gauss, 'k--', linewidth=1., label='Gaussian')
    plt.show()

    plt.figure(figsize=(6,5))
    plt.title('Convergence', fontsize = 18)
    plt.xlabel('Iteration number')
    plt.plot(di['results'][re[0]*10 + re[1]][3], 'b-', label='Misfit value')
    plt.legend(loc=0, fontsize=12, shadow=bool, framealpha=1)
    plt.show()

    x = di['x']/1000.
    y = di['y']/1000.
    z = di['z']/1000.

    for i in range(len(di['results'][re[0]*10 + re[1]][2])):
        di['results'][re[0]*10 + re[1]][2][i].x /= 1000.
        di['results'][re[0]*10 + re[1]][2][i].y /= 1000.
        di['results'][re[0]*10 + re[1]][2][i].z1 /= 1000.
        di['results'][re[0]*10 + re[1]][2][i].z2 /= 1000.


    verts13 = mfun.plot_prisms(di['results'][re[0]*10 + re[1]][2])

    fig = plt.figure(figsize=(10,8))
    ax = fig.add_subplot(111, projection='3d')

    # plot sides
    ax.add_collection3d(Poly3DCollection(verts13, alpha=0.3, 
     facecolor='r', linewidths=0.5, edgecolors='k'))

    ax.set_ylim(np.min(x), np.max(x), 100)
    ax.set_xlim(np.min(y), np.max(y), 100)
    ax.set_zlim(5, -2, 100)
    ax.tick_params(labelsize= 13, pad=8)
    ax.set_ylabel('Y (km)', fontsize= 17)
    ax.set_xlabel('X (km)', fontsize= 17)
    ax.set_zlabel('Z (km)', fontsize= 17)
    #ax.set_xticklabels(np.linspace(-3, 3, 7))
    #ax.set_yticklabels(np.linspace(-3, 3, 7))
    #ax.set_zticklabels(np.linspace(0, 2, 9))
    ax.yaxis.labelpad=13
    ax.zaxis.labelpad=13
    ax.xaxis.labelpad=13
    ax.view_init(10, 120)
    ax.autoscale_view()
    ax.plot_trisurf(di['yt']/1000., di['xt']/1000., -di['topography']/1000., linewidth=0.2, antialiased=True, cmap='inferno')


    plt.show()

IndexError: list index out of range

# Inversion 6

In [49]:
plt.figure(figsize=(6,5))
ax=mpl.subplot(1,1,1)
#plt.title('Residual', fontsize=20)
norm=mpb.colors.Normalize(vmin=-np.max(di['results'][13][-2]), vmax=np.max(di['results'][13][-2]))
plt.tricontourf(di['y'], di['x'], di['results'][13][-2], 20, cmap='RdBu_r', norm=norm).ax.tick_params(labelsize=12)
plt.xlabel('$y$(m)', fontsize=18)
plt.ylabel('$x$(m)', fontsize=18)
clb = plt.colorbar(pad=0.01, aspect=20, shrink=1)
clb.ax.set_title('nT')
mpl.m2km()
clb.ax.tick_params(labelsize=13)
inset = inset_axes(ax, width="40%", height="30%", loc=1, borderpad=0.7)
mean = np.mean(di['results'][13][-2])
std = np.std(di['results'][13][-2])
nbins=30
n, bins, patches = plt.hist(di['results'][13][-2],bins=nbins, normed=True, facecolor='blue')
plt.text(0.4, 0.96, "$\mu$ = {:.2f}\n$\sigma$ = {:.2f}".format(mean, std), 
         horizontalalignment='right',
         verticalalignment='top',
         transform = ax.transAxes, fontsize=10)
gauss = sp.norm.pdf(bins, mean, std)
plt.plot(bins, gauss, 'k--', linewidth=1., label='Gaussian')
plt.show()

In [50]:
plt.figure(figsize=(6,5))
plt.title('Convergence', fontsize = 18)
plt.xlabel('Iteration number')
plt.plot(di['results'][13][3], 'b-', label='Misfit value')
plt.legend(loc=0, fontsize=12, shadow=bool, framealpha=1)
plt.show()

In [51]:
x = di['x']/1000.
y = di['y']/1000.
z = di['z']/1000.

for i in range(len(di['results'][13][2])):
    di['results'][13][2][i].x /= 1000.
    di['results'][13][2][i].y /= 1000.
    di['results'][13][2][i].z1 /= 1000.
    di['results'][13][2][i].z2 /= 1000.
    

In [52]:
verts6 = mfun.plot_prisms(di['results'][13][2])

In [53]:
fig = plt.figure(figsize=(10,8))
ax = fig.add_subplot(111, projection='3d')

# plot sides
ax.add_collection3d(Poly3DCollection(verts6, alpha=0.3, 
 facecolor='r', linewidths=0.3, edgecolors='k'))

ax.set_ylim(np.min(x), np.max(x), 100)
ax.set_xlim(np.min(y), np.max(y), 100)
ax.set_zlim(5, -2, 100)
ax.tick_params(labelsize= 13, pad=8)
ax.set_ylabel('Y (km)', fontsize= 17)
ax.set_xlabel('X (km)', fontsize= 17)
ax.set_zlabel('Z (km)', fontsize= 17)
#ax.set_xticklabels(np.linspace(-3, 3, 7))
#ax.set_yticklabels(np.linspace(-3, 3, 7))
#ax.set_zticklabels(np.linspace(0, 2, 9))
ax.yaxis.labelpad=13
ax.zaxis.labelpad=13
ax.xaxis.labelpad=13
ax.view_init(10, 120)
ax.autoscale_view()

plt.show()

In [102]:
np.linalg.norm(di['results'][6][2][0].props['magnetization'])

6.499999999999999

In [103]:
di['results'][6][2][0].z1

-900.0

In [204]:
plt.figure(figsize=(6,5))
plt.tricontourf(di['yp'], di['xp'],
                polyprism.tf(di['y'], di['xp'], di['zp'], [di['results'][14][2][-1]],
             di['main_field'][0], di['main_field'][1],
            di['results'][14][2][0].props['magnetization']), 20, cmp='RdBu_r')
clb = plt.colorbar(pad=0.01, aspect=20, shrink=1)
clb.ax.set_title('nT')
mpl.m2km()
plt.show()

ValueError: Input arrays xp, yp, and zp must have same shape!

In [202]:
di['y']

array([683000.        , 683204.08163265, 683408.16326531, ...,
       692591.83673469, 692795.91836735, 693000.        ])